In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from gymnasium import spaces
import torch
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
import time




In [ ]:
# https://gymnasium.farama.org/index.html
# !pip install gymnasium
# !pip install torch
# # https://stable-baselines3.readthedocs.io/en/master/guide/install.html
# !pip install stable-baselines3


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
INFO: pip is looking at multiple versions of stable-baselines3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.1.1
    Uninstalling gymnasium-1.1.1:
      Successfully uninstalled gymnasium-1.1.1

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:


class WaterTankEnv(gym.Env):
    """
    A 2D water tank environment where the agent needs to learn to stay in the center.
    
    State space: [x_position, y_position, x_velocity, y_velocity]
    Action space: [thrust_x, thrust_y] - Continuous values between -1 and 1
    """
    
    def __init__(self, tank_size=10.0, max_steps=200, noise_level=0.05):
        super(WaterTankEnv, self).__init__()
        
        # Environment parameters
        self.tank_size = tank_size
        self.max_steps = max_steps
        self.noise_level = noise_level
        
        # Agent dynamics parameters
        self.mass = 1.0
        self.water_resistance = 0.2
        self.max_force = 1.0
        self.dt = 0.1  # time step
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(2,), dtype=np.float32
        )
        self.observation_space = spaces.Box(
            low=np.array([-self.tank_size, -self.tank_size, -5.0, -5.0]),
            high=np.array([self.tank_size, self.tank_size, 5.0, 5.0]),
            dtype=np.float32
        )
        
        # Initialize state
        self.state = None
        self.steps = 0
        self.reset()
        
    def reset(self, seed=None, options=None):
        """Reset the environment to a random position"""
        super().reset(seed=seed)
        
        # Random initial position away from the center
        distance = 0.5 * self.tank_size
        angle = self.np_random.uniform(0, 2 * np.pi)
        x = distance * np.cos(angle)
        y = distance * np.sin(angle)
        
        # Initialize with zero velocity
        self.state = np.array([x, y, 0.0, 0.0], dtype=np.float32)
        self.steps = 0
        
        return self.state, {}
    
    def step(self, action):
        """Take a step in the environment based on the action"""
        self.steps += 1
        
        # Unpack state
        x, y, vx, vy = self.state
        
        # Apply thrust action (with noise to simulate water currents)
        thrust_x = np.clip(action[0], -1.0, 1.0) * self.max_force
        thrust_y = np.clip(action[1], -1.0, 1.0) * self.max_force
        
        # Add environmental noise (water currents)
        thrust_x += self.np_random.uniform(-self.noise_level, self.noise_level)
        thrust_y += self.np_random.uniform(-self.noise_level, self.noise_level)
        
        # Calculate acceleration (F = ma)
        ax = thrust_x / self.mass - self.water_resistance * vx / self.mass
        ay = thrust_y / self.mass - self.water_resistance * vy / self.mass
        
        # Update velocity
        vx += ax * self.dt
        vy += ay * self.dt
        
        # Update position
        x += vx * self.dt
        y += vy * self.dt
        
        # Boundary handling - bounce off walls
        if abs(x) > self.tank_size:
            x = np.sign(x) * self.tank_size
            vx = -vx * 0.9  # Lose some energy on collision
            
        if abs(y) > self.tank_size:
            y = np.sign(y) * self.tank_size
            vy = -vy * 0.9  # Lose some energy on collision
        
        # Update state
        self.state = np.array([x, y, vx, vy], dtype=np.float32)
        
        # Calculate reward
        # Main component: negative distance from center (higher when closer to center)
        distance_from_center = np.sqrt(x**2 + y**2)
        position_reward = -distance_from_center
        
        # Secondary component: penalize high velocities (to encourage stability)
        velocity_penalty = -0.1 * (vx**2 + vy**2)
        
        # Combine rewards
        reward = position_reward + velocity_penalty
        
        # Check if episode is done
        done = self.steps >= self.max_steps
        
        # Additional info
        info = {
            "distance_from_center": distance_from_center,
            "position": (x, y),
            "velocity": (vx, vy)
        }
        
        return self.state, reward, done, False, info
    
    def render(self):
        """Simple text-based rendering"""
        x, y, vx, vy = self.state
        distance = np.sqrt(x**2 + y**2)
        print(f"Position: ({x:.2f}, {y:.2f}), Distance from center: {distance:.2f}")


def visualize_agent(env, model, num_episodes=3):
    """Visualize agent behavior"""
    for episode in range(num_episodes):
        # Initialize plotting
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xlim(-env.tank_size, env.tank_size)
        ax.set_ylim(-env.tank_size, env.tank_size)
        ax.set_xlabel('X position')
        ax.set_ylabel('Y position')
        ax.set_title(f'Agent Trajectory - Episode {episode+1}')
        
        # Draw tank boundary
        tank = plt.Rectangle((-env.tank_size, -env.tank_size), 
                           2*env.tank_size, 2*env.tank_size, 
                           linewidth=2, edgecolor='blue', facecolor='none')
        ax.add_patch(tank)
        
        # Draw center target
        center = plt.Circle((0, 0), 0.5, color='green', alpha=0.3)
        ax.add_patch(center)
        
        # Run episode
        obs, _ = env.reset()
        done = False
        trajectory_x = [obs[0]]
        trajectory_y = [obs[1]]
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _, info = env.step(action)
            trajectory_x.append(obs[0])
            trajectory_y.append(obs[1])
        
        # Plot trajectory
        ax.plot(trajectory_x, trajectory_y, 'r-', alpha=0.7)
        ax.plot(trajectory_x[0], trajectory_y[0], 'bo', label='Start')
        ax.plot(trajectory_x[-1], trajectory_y[-1], 'ro', label='End')
        ax.legend()
        
        plt.grid(True)
        plt.show()


# Create and validate environment
env = WaterTankEnv(tank_size=10.0, max_steps=300, noise_level=0.1)
check_env(env)

# Train the agent
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0003,
    gamma=0.99,  # discount factor
    n_steps=1024,
    batch_size=64,
    n_epochs=10,
    policy_kwargs=dict(
        net_arch=[dict(pi=[64, 64], vf=[64, 64])],
        activation_fn=nn.Tanh
    )
)

# Train
train_timesteps = 100000
model.learn(total_timesteps=train_timesteps)

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Visualize agent behavior
visualize_agent(env, model)

# Test the agent interactively
obs, _ = env.reset()
cumulative_reward = 0
for step in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env.step(action)
    cumulative_reward += reward
    
    # Print current state
    x, y, vx, vy = obs
    distance = np.sqrt(x**2 + y**2)
    print(f"Step {step}: Position ({x:.2f}, {y:.2f}), Distance: {distance:.2f}, Reward: {reward:.2f}")
    
    if done:
        print(f"Episode finished after {step+1} steps. Total reward: {cumulative_reward:.2f}")
        break
    
    time.sleep(0.01)  # Slow down visualization